In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install fastai --upgrade && pip install torch --upgrade

In [ ]:
import fastai
import torch

print(fastai.__version__ , torch.__version__) 

In [ ]:
from fastai.vision.all import *
from ipywidgets.widgets import *

path = "/kaggle/input/pnevmoniya/train"
# fls = get_image_files(path)
# failed = verify_images(path)
# print(len(failed))
# print(failed)

pnevmaniya = DataBlock(
blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.1, seed=22),
    get_y = parent_label,
    item_tfms = Resize(224)
)

dls = pnevmaniya.dataloaders(path)

learn = cnn_learner(dls, resnet34, metrics=accuracy)

learn.fine_tune(3)

In [ ]:
dls.train.show_batch(max_n=32,nrows=4)

In [ ]:
iterp = ClassificationInterpretation.from_learner(learn)
iterp.plot_confusion_matrix()

In [ ]:
iterp.plot_top_losses(5,nrows=1)

In [ ]:
upload = FileUpload()
upload

In [ ]:
img = PILImage.create(upload.data[-1])
pred, prob_id, prob = learn.predict(img)
print("Bashorat: ", pred)
print("ehtimollik", prob[prob_id])

In [ ]:
path_test = "/kaggle/input/pnevmoniya/test"


files = sorted(os.listdir(path_test))
print(files)

# test_set = (path_test).ls().sorted()

In [ ]:
pred_test, prob_test_id, prob_test = learn

In [ ]:
df = pd.read_csv('/kaggle/input/pnevmoniya/sample_solution.csv')
df

In [ ]:
for i in range(0,624):
    label = df.iloc[i,0]
    im = PILImage.create(Path(f"/kaggle/input/pnevmoniya/test/{label}"))
    pred, prob_id, prob = learn.predict(im)
    df.iloc[i,1] = pred

In [ ]:
df

In [ ]:
df['labels'] = df['labels'].map({"PNEUMONIA": 1, "NORMAL": 0})
df

In [ ]:
df.to_csv("submission.csv",index=False)

In [ ]:
learn.export("pnevmaniya_model.pkl")